# Análise exploratória detalhada para compreender estrutura e qualidade do dataset antes de qualquer tratamento.

Foram inspecionadas dimensões, tipos de dados, valores únicos e padrões de agrupamento, incluindo médias de gravidade por tipo de ocorrência e taxas de resolução. Confirmou-se que, exceto gravidade, não há variáveis numéricas adicionais que permitam correlações.

In [14]:
import pandas as pd
import numpy as np
import os

pd.set_option("display.max_columns", None)

In [ ]:
# 1. Carregamento do dataset
# O notebook está em PL001/eda_tratamento_dados
path = "../raw_datasets/ocorrencias_reclamacoes.csv"
if not os.path.exists(path):
    raise FileNotFoundError(f"Ficheiro não encontrado: {path}")

df = pd.read_csv(path)

print("Shape (linhas, colunas):", df.shape)
print("\nInfo geral:")
print(df.info(memory_usage="deep"))
print("\nPrimeiras linhas:")
display(df.head())


Shape (linhas, colunas): (1230, 8)

Info geral:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ocorrencia_id       1230 non-null   object
 1   fração_origem       1230 non-null   object
 2   tipo                1230 non-null   object
 3   descricao_completa  1202 non-null   object
 4   data_ocorrencia     1230 non-null   object
 5   gravidade           1230 non-null   int64 
 6   resolvido           1230 non-null   object
 7   acao_tomada         1212 non-null   object
dtypes: int64(1), object(7)
memory usage: 683.3 KB
None

Primeiras linhas:


,ocorrencia_id,fração_origem,tipo,descricao_completa,data_ocorrencia,gravidade,resolvido,acao_tomada
0,OC00001,FR1553,Ruído,Cão do apartamento 4D ladra constantemente dur...,2025-03-02 16:43,2,True,Morador notificado por escrito. Situação regul...
1,OC00002,FR2061,Estacionamento,"Carro abandonado na garagem há meses, ocupando...",2025-07-19 16:43,4,False,A resolver
2,OC00003,FR1653,Segurança,Pessoas estranhas a entrar no edifício e a per...,2024-08-09 16:43,4,False,em analise
3,OC00004,FR1001,Vandalismo,Grafittis na entrada do bloco norte. Necessári...,2024-11-10 16:43,4,True,Limpeza realizada. Apresentada queixa na políc...
4,OC00005,FR0403,Ruído,Barulho excessivo proveniente do apartamento 5...,2025-05-15 16:43,4,False,Pendente


In [23]:
print("\nTipos de dados por coluna (antes das conversões):")
print(df.dtypes)


Tipos de dados por coluna (antes das conversões):
ocorrencia_id         object
fração_origem         object
tipo                  object
descricao_completa    object
data_ocorrencia       object
gravidade              int64
resolvido             object
acao_tomada           object
dtype: object


In [ ]:
# 3. Conversões segundo o schema oficial
df['data_ocorrencia'] = pd.to_datetime(df['data_ocorrencia'], errors='coerce')
df['tipo'] = df['tipo'].astype('category')
df['fração_origem'] = df['fração_origem'].astype('category')
df['resolvido'] = df['resolvido'].astype(str).str.lower().map(
    {'sim': True, 'não': False, 'nao': False, 'true': True, 'false': False}
)

print("\nTipos de dados após conversões:")
print(df.dtypes)


Tipos de dados após conversões:
ocorrencia_id                 object
fração_origem               category
tipo                        category
descricao_completa            object
data_ocorrencia       datetime64[ns]
gravidade                      int64
resolvido                     object
acao_tomada                   object
dtype: object


In [ ]:
# 3. Descrição estatística
print("\nDescrição estatística global:")
display(df.describe(include='all'))


Descrição estatística global:


,ocorrencia_id,fração_origem,tipo,descricao_completa,data_ocorrencia,gravidade,resolvido,acao_tomada
count,1230,1230,1230,1202,1179,1230.000000,1199,1212
unique,1200,940,27,113,NaN,NaN,2,10
top,OC00305,FR1510,Ruído,Mancha de humidade no teto da sala com gotejam...,NaN,NaN,False,Morador notificado por escrito. Situação regul...
freq,2,4,358,107,NaN,NaN,623,198
mean,NaN,NaN,NaN,NaN,2024-12-23 23:41:55.877862656,2.904878,NaN,NaN
min,NaN,NaN,NaN,NaN,2024-03-21 16:43:00,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,2024-08-11 04:43:00,2.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,2024-12-25 16:43:00,3.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,2025-05-10 04:43:00,4.000000,NaN,NaN
max,NaN,NaN,NaN,NaN,2025-09-12 16:43:00,10.000000,NaN,NaN


In [ ]:
# 4. Cardinalidade de variáveis categóricas
print("\nCardinalidade (número de valores únicos) nas colunas categóricas:")
for col in df.select_dtypes(include=['category','object']).columns:
    print(f" - {col}: {df[col].nunique()} valores únicos")


Cardinalidade (número de valores únicos) nas colunas categóricas:
 - ocorrencia_id: 1200 valores únicos
 - fração_origem: 940 valores únicos
 - tipo: 27 valores únicos
 - descricao_completa: 113 valores únicos
 - resolvido: 2 valores únicos
 - acao_tomada: 10 valores únicos


In [ ]:
# 5. Padrões com groupby
print("\nPadrões por tipo de ocorrência:")
print(df.groupby("tipo").size().sort_values(ascending=False))

print("\nMédia da gravidade por tipo de ocorrência:")
print(df.groupby("tipo")["gravidade"].mean().sort_values(ascending=False))

print("\nTaxa de resolução por tipo:")
print(df.groupby("tipo")["resolvido"].mean().sort_values(ascending=False))


Padrões por tipo de ocorrência:
tipo
Ruído             358
Infiltração       212
Limpeza           165
Segurança         163
Vandalismo        124
Estacionamento    112
ruido              14
Ruido               8
RUÍDO               8
infiltracao         6
VANDALISMO          6
vandalismo          6
Humidade            5
limpeza             5
SEGURANÇA           4
INFILTRAÇÃO         4
estacionamento      4
LIMPEZA             3
Seguranca           3
Vandalism           3
Higiene             3
seguranca           3
Barulho             3
Parking             2
ESTACIONAMENTO      2
Infiltraçao         2
Garagem             2
dtype: int64

Média da gravidade por tipo de ocorrência:
tipo
INFILTRAÇÃO       3.750000
SEGURANÇA         3.750000
seguranca         3.666667
Seguranca         3.666667
ESTACIONAMENTO    3.500000
RUÍDO             3.375000
infiltracao       3.333333
VANDALISMO        3.333333
estacionamento    3.250000
vandalismo        3.166667
Estacionamento    3.071429
ruido    

/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2265/991572426.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("tipo").size().sort_values(ascending=False))
/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2265/991572426.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("tipo")["gravidade"].mean().sort_values(ascending=False))
/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2265/991572426.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain

In [ ]:
# 6. Estatísticas segmentadas
print("\nGravidade por fração de origem (primeiros 10):")
print(df.groupby("fração_origem")["gravidade"].describe().head(10))


Gravidade por fração de origem (primeiros 10):


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2265/1584680951.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("fração_origem")["gravidade"].describe().head(10))


               count      mean       std  min   25%  50%   75%  max
fração_origem                                                      
FR0001           2.0  1.500000  0.707107  1.0  1.25  1.5  1.75  2.0
FR0002           1.0  4.000000       NaN  4.0  4.00  4.0  4.00  4.0
FR0003           1.0  4.000000       NaN  4.0  4.00  4.0  4.00  4.0
FR0004           1.0  4.000000       NaN  4.0  4.00  4.0  4.00  4.0
FR0006           3.0  3.666667  0.577350  3.0  3.50  4.0  4.00  4.0
FR0008           1.0  3.000000       NaN  3.0  3.00  3.0  3.00  3.0
FR0013           1.0  2.000000       NaN  2.0  2.00  2.0  2.00  2.0
FR0015           2.0  3.000000  0.000000  3.0  3.00  3.0  3.00  3.0
FR0016           2.0  3.000000  0.000000  3.0  3.00  3.0  3.00  3.0
FR0017           1.0  2.000000       NaN  2.0  2.00  2.0  2.00  2.0


In [35]:
# 7. Correlações numéricas
print("\nMatriz de correlação entre variáveis numéricas:")
num_cols = df.select_dtypes(include=[np.number]).columns
if len(num_cols) > 1:
    display(df[num_cols].corr())
else:
    print("Poucas colunas numéricas para correlação.")


Matriz de correlação entre variáveis numéricas:
Poucas colunas numéricas para correlação.


 - Distribuição dos tipos de ocorrência
 - Gravidade média por tipo
 - Percentagem de casos resolvidos
 - Possíveis relações entre gravidade e resolução